## 간단한 RAG에서 청크 크기 평가하기

검색 증강 생성(RAG) 파이프라인에서 올바른 청크 크기를 선택하는 것은 검색 정확도를 향상시키는 데 매우 중요합니다. 목표는 검색 성능과 응답 품질 사이의 균형을 맞추는 것입니다.

이 섹션에서는 다음과 같은 방법으로 다양한 청크 크기를 평가합니다:

1. PDF에서 텍스트 추출하기.
2. 텍스트를 다양한 크기의 청크로 분할하기.
3. 각 청크에 대한 임베딩 생성하기.
4. 질의에 대한 관련 청크 검색하기.
5. 검색된 청크를 사용하여 응답 생성하기.
6. 충실도(Faithfulness)와 관련성(Relevancy) 평가하기.
7. 다양한 청크 크기에 대한 결과 비교하기.

---#### 초보자를 위한 추가 설명
**청크 크기는 왜 중요할까요?**
청크 크기는 RAG 시스템의 성능에 직접적인 영향을 미치는 중요한 요소입니다. 어떤 크기가 가장 좋은지는 정해져 있지 않으며, 데이터의 종류와 사용자의 질문 유형에 따라 달라집니다.
*   **작은 청크 (예: 128)**: 
    *   **장점**: 매우 구체적이고 집중된 정보를 담고 있어, 특정 사실(Fact)에 대한 질문에 답할 때 정확도가 높을 수 있습니다. 관련 없는 내용이 섞일 가능성이 적습니다.
    *   **단점**: 전체적인 맥락을 잃어버리기 쉽습니다. 여러 문장에 걸쳐 설명되는 복잡한 개념을 이해하기 어려울 수 있습니다.
*   **큰 청크 (예: 512)**: 
    *   **장점**: 더 넓은 맥락을 포함하므로, 요약이나 복잡한 주제에 대한 질문에 더 나은 답변을 제공할 수 있습니다.
    *   **단점**: 관련 없는 정보(노이즈)가 포함될 가능성이 커져, 오히려 LLM이 정확한 답변을 생성하는 데 방해가 될 수 있습니다.

이 노트북에서는 다양한 크기의 청크를 만들어보고, 어떤 크기가 주어진 질문에 대해 가장 '충실하고' '관련성 높은' 답변을 만드는지 실험해 봅니다.

## 환경 설정
필요한 라이브러리를 가져옵니다.

In [1]:
import fitz         # PyMuPDF 라이브러리, PDF 파일 처리를 위해 사용
import os           # 운영체제와 상호작용하기 위한 라이브러리 (예: 환경 변수 접근)
import numpy as np  # 수치 연산을 위한 라이브러리
import json         # JSON 파일 처리를 위한 라이브러리
from openai import OpenAI # OpenAI API 클라이언트

## OpenAI API 클라이언트 설정
임베딩과 응답 생성을 위해 OpenAI 클라이언트를 초기화합니다.

In [ ]:
# 기본 URL과 API 키로 OpenAI 클라이언트 초기화
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 검색
)

## PDF에서 텍스트 추출
먼저 `AI_Information.pdf` 파일에서 텍스트를 추출합니다.

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
    pdf_path (str): PDF 파일 경로.

    Returns:
    str: PDF에서 추출된 텍스트.
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 텍스트를 저장할 빈 문자열 초기화
    
    # PDF의 각 페이지를 순회
    for page in mypdf:
        # 현재 페이지에서 텍스트를 추출하고 공백 추가
        all_text += page.get_text("text") + " "

    # 앞뒤 공백을 제거한 추출된 텍스트 반환
    return all_text.strip()

# PDF 파일 경로 정의
pdf_path = "data/AI_Information.pdf"

# PDF 파일에서 텍스트 추출
extracted_text = extract_text_from_pdf(pdf_path)

# 추출된 텍스트의 첫 500자 출력
print(extracted_text[:500])

Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past f


## 추출된 텍스트 청킹하기
검색 성능을 향상시키기 위해, 추출된 텍스트를 다양한 크기의 중첩된 청크로 분할합니다.

In [4]:
def chunk_text(text, n, overlap):
    """
    텍스트를 중첩된 청크로 분할합니다.

    Args:
    text (str): 청킹할 텍스트.
    n (int): 청크당 문자 수.
    overlap (int): 청크 간 중첩되는 문자 수.

    Returns:
    List[str]: 텍스트 청크 리스트.
    """
    chunks = []  # 청크를 저장할 빈 리스트 초기화
    for i in range(0, len(text), n - overlap):
        # 현재 인덱스부터 인덱스 + 청크 크기까지의 텍스트 청크를 추가
        chunks.append(text[i:i + n])
    
    return chunks  # 텍스트 청크 리스트 반환

# 평가할 다양한 청크 크기 정의
chunk_sizes = [128, 256, 512]

# 각 청크 크기별 텍스트 청크를 저장할 딕셔너리 생성
text_chunks_dict = {size: chunk_text(extracted_text, size, size // 5) for size in chunk_sizes}

# 각 청크 크기별로 생성된 청크 수 출력
for size, chunks in text_chunks_dict.items():
    print(f"청크 크기: {size}, 청크 수: {len(chunks)}")

청크 크기: 128, 청크 수: 326
청크 크기: 256, 청크 수: 164
청크 크기: 512, 청크 수: 82


## 텍스트 청크에 대한 임베딩 생성
임베딩은 유사도 검색을 위해 텍스트를 숫자 표현으로 변환합니다.

In [5]:
from tqdm import tqdm

def create_embeddings(texts, model="BAAI/bge-en-icl"):
    """
    텍스트 리스트에 대한 임베딩을 생성합니다.

    Args:
    texts (List[str]): 입력 텍스트 리스트.
    model (str): 임베딩 모델.

    Returns:
    List[np.ndarray]: 숫자 임베딩 리스트.
    """
    # 지정된 모델을 사용하여 임베딩 생성
    response = client.embeddings.create(model=model, input=texts)
    # 응답을 numpy 배열 리스트로 변환하여 반환
    return [np.array(embedding.embedding) for embedding in response.data]

# 각 청크 크기별 임베딩 생성
# text_chunks_dict의 각 청크 크기와 해당 청크들을 순회
chunk_embeddings_dict = {size: create_embeddings(chunks) for size, chunks in tqdm(text_chunks_dict.items(), desc="임베딩 생성 중")}

임베딩 생성 중: 100%|██████████| 3/3 [00:11<00:00,  3.71s/it]


## 시맨틱 검색 수행
사용자 질의에 가장 관련성 높은 텍스트 청크를 찾기 위해 코사인 유사도를 사용합니다.

In [6]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터.
    vec2 (np.ndarray): 두 번째 벡터.

    Returns:
    float: 코사인 유사도 점수.
    """

    # 두 벡터의 내적 계산
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [7]:
def retrieve_relevant_chunks(query, text_chunks, chunk_embeddings, k=5):
    """
    상위 k개의 가장 관련성 높은 텍스트 청크를 검색합니다.
    
    Args:
    query (str): 사용자 질의.
    text_chunks (List[str]): 텍스트 청크 리스트.
    chunk_embeddings (List[np.ndarray]): 텍스트 청크의 임베딩.
    k (int): 반환할 상위 청크 수.
    
    Returns:
    List[str]: 가장 관련성 높은 텍스트 청크.
    """
    # 질의에 대한 임베딩 생성 - 질의를 리스트로 전달하고 첫 번째 항목을 가져옴
    query_embedding = create_embeddings([query])[0]
    
    # 질의 임베딩과 각 청크 임베딩 간의 코사인 유사도 계산
    similarities = [cosine_similarity(query_embedding, emb) for emb in chunk_embeddings]
    
    # 상위 k개의 가장 유사한 청크의 인덱스 가져오기
    top_indices = np.argsort(similarities)[-k:][::-1]
    
    # 상위 k개의 가장 관련성 높은 텍스트 청크 반환
    return [text_chunks[i] for i in top_indices]

In [8]:
# JSON 파일에서 검증 데이터 로드
with open('data/val.json') as f:
    data = json.load(f)

# 검증 데이터에서 네 번째 질문 추출
query = data[3]['question']

# 각 청크 크기별로 관련 청크 검색
retrieved_chunks_dict = {size: retrieve_relevant_chunks(query, text_chunks_dict[size], chunk_embeddings_dict[size]) for size in chunk_sizes}

# 청크 크기 256에 대해 검색된 청크 출력
print(retrieved_chunks_dict[256])

['AI enables personalized medicine by analyzing individual patient data, predicting treatment \nresponses, and tailoring interventions. Personalized medicine enhances treatment effectiveness \nand reduces adverse effects. \nRobotic Surgery \nAI-powered robotic s', ' analyzing biological data, predicting drug \nefficacy, and identifying potential drug candidates. AI-powered systems reduce the time and cost \nof bringing new treatments to market. \nPersonalized Medicine \nAI enables personalized medicine by analyzing indiv', 'g \npatient outcomes, and assisting in treatment planning. AI-powered tools enhance accuracy, \nefficiency, and patient care. \nDrug Discovery and Development \nAI accelerates drug discovery and development by analyzing biological data, predicting drug \neffica', 'mains. \nThese applications include: \nHealthcare \nAI is transforming healthcare through applications such as medical diagnosis, drug discovery, \npersonalized medicine, and robotic surgery. AI-powered to

## 검색된 청크를 기반으로 응답 생성
청크 크기 `256`에 대해 검색된 텍스트를 기반으로 응답을 생성해 보겠습니다.

In [9]:
# AI 어시스턴트에 대한 시스템 프롬프트 정의
system_prompt = "당신은 주어진 컨텍스트를 기반으로만 엄격하게 답변하는 AI 어시스턴트입니다. 제공된 컨텍스트에서 직접적으로 답변을 도출할 수 없는 경우, '답변하기에 충분한 정보가 없습니다.'라고 응답하세요."

def generate_response(query, system_prompt, retrieved_chunks, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    검색된 청크를 기반으로 AI 응답을 생성합니다.

    Args:
    query (str): 사용자 질의.
    retrieved_chunks (List[str]): 검색된 텍스트 청크 리스트.
    model (str): AI 모델.

    Returns:
    str: AI가 생성한 응답.
    """
    # 검색된 청크를 단일 컨텍스트 문자열로 결합
    context = "\n".join([f"컨텍스트 {i+1}:\n{chunk}" for i, chunk in enumerate(retrieved_chunks)])
    
    # 컨텍스트와 질의를 결합하여 사용자 프롬프트 생성
    user_prompt = f"{context}\n\n질문: {query}"

    # 지정된 모델을 사용하여 AI 응답 생성
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # AI 응답의 내용 반환
    return response.choices[0].message.content

AI contributes to personalized medicine by analyzing individual patient data, predicting treatment responses, and tailoring interventions. This enables personalized medicine to enhance treatment effectiveness and reduce adverse effects.


## AI 응답 평가
강력한 LLM을 사용하여 충실도와 관련성을 기준으로 응답에 점수를 매깁니다.

In [10]:
# 평가 점수 시스템 상수 정의
SCORE_FULL = 1.0     # 완전 일치 또는 완전히 만족
SCORE_PARTIAL = 0.5  # 부분 일치 또는 다소 만족
SCORE_NONE = 0.0     # 불일치 또는 불만족

In [11]:
# 엄격한 평가 프롬프트 템플릿 정의
FAITHFULNESS_PROMPT_TEMPLATE = """
AI 응답이 실제 답변과 비교하여 얼마나 충실한지 평가하세요.
사용자 질의: {question}
AI 응답: {response}
실제 답변: {true_answer}

충실도는 AI 응답이 환각(hallucination) 없이 실제 답변의 사실과 얼마나 잘 일치하는지를 측정합니다.

지침:
- 다음 값만을 사용하여 엄격하게 점수를 매기세요:
    * {full} = 완전히 충실함, 실제 답변과 모순 없음
    * {partial} = 부분적으로 충실함, 사소한 모순 있음
    * {none} = 충실하지 않음, 주요 모순 또는 환각 있음
- 설명이나 추가 텍스트 없이 숫자 점수({full}, {partial}, 또는 {none})만 반환하세요.
"""

In [12]:
RELEVANCY_PROMPT_TEMPLATE = """
사용자 질의에 대한 AI 응답의 관련성을 평가하세요.
사용자 질의: {question}
AI 응답: {response}

관련성은 응답이 사용자의 질문을 얼마나 잘 다루는지를 측정합니다.

지침:
- 다음 값만을 사용하여 엄격하게 점수를 매기세요:
    * {full} = 완전히 관련성 높음, 질의를 직접적으로 다룸
    * {partial} = 부분적으로 관련성 있음, 일부 측면을 다룸
    * {none} = 관련성 없음, 질의를 다루지 못함
- 설명이나 추가 텍스트 없이 숫자 점수({full}, {partial}, 또는 {none})만 반환하세요.
"""

In [13]:
def evaluate_response(question, response, true_answer):
        """
        충실도와 관련성을 기반으로 AI 생성 응답의 품질을 평가합니다.

        Args:
        question (str): 사용자의 원본 질문.
        response (str): 평가 대상인 AI 생성 응답.
        true_answer (str): 정답으로 사용되는 실제 답변.

        Returns:
        Tuple[float, float]: (충실도_점수, 관련성_점수)를 포함하는 튜플.
                                                각 점수는 1.0 (전체), 0.5 (부분), 또는 0.0 (없음) 중 하나입니다.
        """
        # 평가 프롬프트 포맷팅
        faithfulness_prompt = FAITHFULNESS_PROMPT_TEMPLATE.format(
                question=question, 
                response=response, 
                true_answer=true_answer,
                full=SCORE_FULL,
                partial=SCORE_PARTIAL,
                none=SCORE_NONE
        )
        
        relevancy_prompt = RELEVANCY_PROMPT_TEMPLATE.format(
                question=question, 
                response=response,
                full=SCORE_FULL,
                partial=SCORE_PARTIAL,
                none=SCORE_NONE
        )

        # 모델에 충실도 평가 요청
        faithfulness_response = client.chat.completions.create(
               model="meta-llama/Llama-3.2-3B-Instruct",
                temperature=0,
                messages=[
                        {"role": "system", "content": "당신은 객관적인 평가자입니다. 숫자 점수만 반환하세요."},
                        {"role": "user", "content": faithfulness_prompt}
                ]
        )
        
        # 모델에 관련성 평가 요청
        relevancy_response = client.chat.completions.create(
                model="meta-llama/Llama-3.2-3B-Instruct",
                temperature=0,
                messages=[
                        {"role": "system", "content": "당신은 객관적인 평가자입니다. 숫자 점수만 반환하세요."},
                        {"role": "user", "content": relevancy_prompt}
                ]
        )
        
        # 점수 추출 및 잠재적 파싱 오류 처리
        try:
                faithfulness_score = float(faithfulness_response.choices[0].message.content.strip())
        except ValueError:
                print("경고: 충실도 점수를 파싱할 수 없어 기본값 0으로 설정합니다.")
                faithfulness_score = 0.0
                
        try:
                relevancy_score = float(relevancy_response.choices[0].message.content.strip())
        except ValueError:
                print("경고: 관련성 점수를 파싱할 수 없어 기본값 0으로 설정합니다.")
                relevancy_score = 0.0

        return faithfulness_score, relevancy_score

# 첫 번째 검증 데이터에 대한 실제 답변
true_answer = data[3]['ideal_answer']

# 청크 크기 256과 128에 대한 응답 평가
faithfulness, relevancy = evaluate_response(query, ai_responses_dict[256], true_answer)
faithfulness2, relevancy2 = evaluate_response(query, ai_responses_dict[128], true_answer)

# 평가 점수 출력
print(f"충실도 점수 (청크 크기 256): {faithfulness}")
print(f"관련성 점수 (청크 크기 256): {relevancy}")

print(f"\n")

print(f"충실도 점수 (청크 크기 128): {faithfulness2}")
print(f"관련성 점수 (청크 크기 128): {relevancy2}")

충실도 점수 (청크 크기 256): 0.5
관련성 점수 (청크 크기 256): 0.5


충실도 점수 (청크 크기 128): 0.5
관련성 점수 (청크 크기 128): 0.5
